In [ ]:
## data folder
## /content/drive/MyDrive/work_chalenges/cazton/data

In [1]:
!pip install requests beautifulsoup4

In [2]:
import requests
from bs4 import BeautifulSoup
import json
import os
import re

In [ ]:
def get_page_links(url, base_url):

    try:

        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        links = set()

        for link in soup.find_all('a'):

            href = link.get('href')

            if href and href.startswith('/'):
                links.add(base_url + href)

        return links

    except requests.exceptions.RequestException:
        return set()

def scrape_page(url):
    # print('Trying to scrape - ', url)

    try:

        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        # print("Scraped")

        return soup.get_text()

    except requests.exceptions.RequestException:
        return ""

def save_data(data, path):

    os.makedirs(os.path.dirname(path), exist_ok = True)

    with open(path, 'w') as file:
        json.dump(data, file, indent = 2)

In [ ]:
base_url = 'https://cazton.com'
visited = set()
to_visit = {base_url}

while to_visit:

    current_url = to_visit.pop()

    if current_url in visited:
        continue

    visited.add(current_url)

    content = scrape_page(current_url)
    if content:

        file_name = current_url.replace(base_url, '').replace('/', '__') + '.json'
        if file_name == '.json':
          file_name = 'base.json'

        save_data({'url': current_url, 'content': content}, '/content/drive/MyDrive/work_chalenges/cazton/data/' + file_name)

    page_links = get_page_links(current_url, base_url)
    to_visit.update(page_links - visited)

In [ ]:
!ls /content/drive/MyDrive/work_chalenges/cazton/data

In [ ]:
complete_data = dict()

for json_file in os.listdir('/content/drive/MyDrive/work_chalenges/cazton/data'):

  file_path = '/content/drive/MyDrive/work_chalenges/cazton/data/' + json_file

  with open(file_path, 'r', encoding = 'utf-8') as file:
    data = json.load(file)

  if 'content' in data:
    complete_data[json_file[:-5]] = re.sub(r'\n+', '\n', data['content'])

with open('/content/drive/MyDrive/work_chalenges/cazton/full_data.json', 'w') as file:
  json.dump(complete_data, file, indent = 2)

In [23]:
train_data = list()

for idx, item in enumerate(compelete_data):

  if i != 'base':
    ques = "tell me about cazton in the context of" + i.replace('_', ' ')

  else:
    ques = "tell me about cazton"

  data_point = dict()
  data_point['question'] = ques
  data_point['context'] = compelete_data[item]
  data_point['answers'] = dict()
  data_point['answers']['answer_start'] = [0]
  data_point['answers']['text'] = compelete_data[item]

  train_data.append(data_point)

In [24]:
with open('/content/drive/MyDrive/work_chalenges/cazton/train_data.json', 'w') as file:
  json.dump(train_data, file, indent = 2)

In [19]:
test_data = list()

for idx, item in enumerate(list(compelete_data.keys())[:10]):

  if i != 'base':
    ques = "tell me about cazton in the context of" + i.replace('_', ' ')

  else:
    ques = "tell me about cazton"

  data_point = dict()
  data_point['question'] = ques
  data_point['context'] = compelete_data[item]
  data_point['answers'] = dict()
  data_point['answers']['answer_start'] = [0]
  data_point['answers']['text'] = compelete_data[item]

  test_data.append(data_point)

In [20]:
with open('/content/drive/MyDrive/work_chalenges/cazton/test_data.json', 'w') as file:
  json.dump(test_data, file, indent = 2)

In [ ]:
{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame'
}

In [ ]:
!pip install transformers datasets evaluate

In [21]:
from datasets import load_dataset
dataset = load_dataset("json", data_files = '/content/drive/MyDrive/work_chalenges/cazton/test_data.json')
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['context', 'answers', 'question'],
        num_rows: 10
    })
})

In [22]:
dataset['train'][0]

{'context': "\nConsulting, Training, Recruiting | Cazton\n(512) 318-2336\ninfo@cazton.com\nToggle navigation\nHome\nConsulting\nCustom Software Development Services\nBig Data & AI\nAI Express PoC Service\nOpenAI\nAI Agents\nOpenAI Case Studies\nAzure OpenAI\nChatGPT for Business\nTech Debt Terminators\nPyTorch\nMicrosoft Fabric\nTensorFlow\nArtificial Intelligence\nBig Data\nOpenAI vs CaztonAI\nDatabricks\nApache Spark\nHadoop & HDFS\nApache Kafka\nSpark.NET\nCloud & DevOps\nAmazon Web Services\nAzure\nGoogle Cloud Platform\nSnowflake\nDevOps\nMicroservices\nDocker\nKubernetes\nTerraform\nAgile\nDatabase Development\nPolyglot Persistence\nSQL Server\nPostGres\nAzure Cosmos DB\nMongoDB\nCassandra\nRedis\nApache Ignite\nWeb, Mobile & Desktop\n.NET 8\n.NET 7\nJava\n.NET 6\nReact\nReact Native (iOS & Android)\nIonic (iOS & Android)\nAngular\nBlazor\nSwift (iOS)\nASP.NET Core\nNode.js\nProgressive Web Apps\nElectron.js (Mac, Windows & Linux)\n.NET Core\n.NET\nEF Core\nTypeScript\nMicrosoft 